In [1]:
# Импортируем модули и настраиваем пути к данным и хранилищу Qdrant.
import os
from pathlib import Path
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder  # [CHANGE] CrossEncoder используем для реранкера
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
from uuid import uuid4  # [CHANGE] uuid для универсальных идентификаторов
import torch

# Ищем корневую папку проекта, чтобы корректно находить данные.
BASE_PATH = Path.cwd()
if not (BASE_PATH / 'data').exists():
    for parent in BASE_PATH.parents:
        if (parent / 'data').exists():
            BASE_PATH = parent
            break

DATA_PATH = BASE_PATH / 'data' / 'big_data.csv'
QDRANT_PATH = BASE_PATH / 'qdrant_storage'
COLLECTION_NAME = 'books_by_annotation'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Используемое устройство: {device}')
print(f'Файл с данными: {DATA_PATH}')


Используемое устройство: cuda
Файл с данными: c:\Users\user\ElbrusBootcamp\2nd_phase_SD\find_my_book\data\big_data.csv


In [2]:
# Загружаем таблицу с книгами, оставляем нужные столбцы и удаляем пустые строки.
df = pd.read_csv(DATA_PATH, encoding='utf-8')
df = df[['title', 'author', 'annotation', 'page_url', 'image_url']].dropna(subset=['annotation'])
df = df.fillna('')
df = df.drop_duplicates(subset=['annotation']).reset_index(drop=True)
df['uuid'] = [str(uuid4()) for _ in range(len(df))]  # [CHANGE] выдаём каждой книге собственный UUID
print(f'Количество книг после очистки: {len(df)}')
df.head(2)

Количество книг после очистки: 28458


,title,author,annotation,page_url,image_url,uuid
0,ИСТОРИК №09 (129). Сентябрь 2025,Без автора,Читайте в номере: ПУТЬ К ЦАРСТВУ: как Москва и...,https://www.biblio-globus.ru/product/11060070,https://static1.bgshop.ru/imagehandler.ashx?fi...,ef4397b5-c46e-4ada-8bbe-209f4814f4c2
1,Тайны XIX века. Тайны морей и островов,Булычев К.,"Прошлое надежно хранит свои тайны и, как всем ...",https://www.biblio-globus.ru/product/11021457,https://static1.bgshop.ru/imagehandler.ashx?fi...,2049ef7f-c0c4-4a1f-bd06-edf357dba262


In [3]:
# Преобразуем таблицу в список словарей, чтобы позже легко сохранять текст в Qdrant.
documents = df.to_dict(orient='records')  # [CHANGE] в каждом словаре остаётся uuid
texts = [doc['annotation'] for doc in documents]
doc_inputs = [f"passage: {text}" for text in texts]  # [CHANGE] форматируем аннотации под модель e5
print(f'Пример аннотации: {texts[0][:120]}...')


Пример аннотации: Читайте в номере: ПУТЬ К ЦАРСТВУ: как Москва из центра маленького княжества превратилась в столицу огромного Русского го...


In [4]:
# [CHANGE] Загружаем модель эмбеддингов d0rj/e5-base-en-ru.
# У неё особый формат входа: запросы должны начинаться с 'query:', а документы с 'passage:'.
model_name = 'd0rj/e5-base-en-ru'
print(f'Выбранная модель эмбеддингов: {model_name}')

embedding_model = SentenceTransformer(
    model_name,
    device=device
)

embedding_model.max_seq_length = 512  # [CHANGE] ограничиваем длину последовательности для стабильности
embedding_model.tokenizer.model_max_length = 512  # [CHANGE] синхронизируем ограничение у токенизатора
# embedding_model.tokenizer.truncation_side = 'right'  # [CHANGE] отрезаем лишнее справа, чтобы не терять начало аннотации


Выбранная модель эмбеддингов: d0rj/e5-base-en-ru


In [5]:
# [CHANGE] Загружаем реранкер, который переоценивает кандидатов по запросу.
reranker_name = 'qilowoq/bge-reranker-v2-m3-en-ru'
print(f'Подключаем реранкер: {reranker_name}')
reranker = CrossEncoder(
    reranker_name,
    device=device
)


Подключаем реранкер: qilowoq/bge-reranker-v2-m3-en-ru


In [9]:
# Считаем вектора для аннотаций в формате 'passage:' и сохраним их в память.
annotation_vectors = embedding_model.encode(
    doc_inputs,
    batch_size=64,  # [CHANGE] уменьшаем батч, чтобы не перегружать GPU
    convert_to_numpy=True,
    show_progress_bar=True,
    normalize_embeddings=True,  # [CHANGE] e5 рекомендует L2-нормализацию
    precision='float32'  # [CHANGE] принудительно считаем в FP32 без смешанной точности
)
print(f'Размер векторов: {annotation_vectors.shape[1]}')


Batches:   0%|          | 0/445 [00:00<?, ?it/s]

Размер векторов: 768


## Загрузка БД ниже, если она уже есть

In [10]:
# Создаем или очищаем локальную коллекцию Qdrant, куда будем складывать вектора.
qdrant_client = QdrantClient(path=QDRANT_PATH)
vector_size = int(annotation_vectors.shape[1])

qdrant_client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=qmodels.VectorParams(
        size=vector_size,
        distance=qmodels.Distance.COSINE
    )
)

print('Коллекция готова:', COLLECTION_NAME)

Коллекция готова: books_by_annotation


C:\Users\user\AppData\Local\Temp\ipykernel_6128\327398713.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


In [11]:
# Загружаем вектора и связанные данные в Qdrant.
payloads = [
    {
        'title': doc['title'],
        'author': doc['author'],
        'annotation': doc['annotation'],
        'page_url': doc['page_url'],
        'image_url': doc['image_url'],
        'uuid': doc['uuid']  # [CHANGE] сохраняем uuid вместе с остальными данными
    }
    for doc in documents
]

qdrant_client.upload_collection(
    collection_name=COLLECTION_NAME,
    vectors=annotation_vectors,
    payload=payloads,
    ids=[doc['uuid'] for doc in documents],  # [CHANGE] используем uuid как идентификаторы
    parallel=1,
    batch_size=256
)

print('Загрузили объектов:', len(payloads))

Загрузили объектов: 28458


C:\Users\user\AppData\Local\Temp\ipykernel_6128\1904896058.py:14: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 28458 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  qdrant_client.upload_collection(


### Загрузка тут ↓

In [2]:
# [CHANGE] Подключаемся к сохранённой коллекции Qdrant, чтобы не пересоздавать её.
qdrant_client = QdrantClient(path=QDRANT_PATH)

try:
    collection_info = qdrant_client.get_collection(COLLECTION_NAME)
    print(f"Коллекция '{COLLECTION_NAME}' найдена.")
    print(f"Количество векторов: {collection_info.points_count}")
except Exception as error:
    print(f"Коллекция '{COLLECTION_NAME}' не найдена. Запустите ячейки с подготовкой данных.")
    print(f'Подробности: {error}')

Коллекция 'books_by_annotation' найдена.
Количество векторов: 28458


C:\Users\user\AppData\Local\Temp\ipykernel_4056\2620968451.py:2: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <books_by_annotation> contains 28458 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  qdrant_client = QdrantClient(path=QDRANT_PATH)


In [7]:
# [CHANGE] Обновляем поиск: эмбеддинги считаем через e5, затем реранкуем список кандидатов.
def search_books(query: str, top_k: int = 3, fetch_limit: int = 15):
    fetch_limit = max(fetch_limit, top_k)  # [CHANGE] убеждаемся, что кандидатов не меньше итогового числа
    query_input = f"query: {query}"  # [CHANGE] формат запроса для e5
    query_vector = embedding_model.encode(
        query_input,
        convert_to_numpy=True,
        normalize_embeddings=True  # [CHANGE] держим формат одинаковым с документами
    )
    hits = qdrant_client.search(
        collection_name=COLLECTION_NAME,
        query_vector=query_vector,
        limit=fetch_limit
    )
    if not hits:
        return []

    pairs = [[query, hit.payload['annotation']] for hit in hits]  # [CHANGE] реранкеру хватает исходного текста запроса
    rerank_scores = reranker.predict(pairs)

    results = []
    for hit, rerank_score in zip(hits, rerank_scores):
        payload = hit.payload
        results.append({
            'uuid': payload.get('uuid', ''),
            'title': payload['title'],
            'author': payload['author'],
            'annotation': payload['annotation'],
            'page_url': payload['page_url'],
            'image_url': payload.get('image_url', ''),
            'search_score': round(hit.score, 4),
            'rerank_score': float(rerank_score)  # [CHANGE] сохраняем оценку реранкера
        })

    results.sort(key=lambda item: item['rerank_score'], reverse=True)  # [CHANGE] сортируем по реранкеру
    return results[:top_k]

print('Поиск готов: используем e5-эмбеддинги + реранкер.')


Поиск готов: используем e5-эмбеддинги + реранкер.


In [8]:
# Проверяем пайплайн: вводим запрос, запускаем поиск и выводим три подходящие книги.
user_query = 'Хочу смеяться пять минут'
found_books = search_books(user_query, top_k=5)

if not found_books:
    print('Ничего не нашли, попробуйте изменить запрос.')
else:
    for idx, book in enumerate(found_books, start=1):
        print(f"{idx}. {book['title']} - {book['author']}")
        print(f"   UUID: {book['uuid']}")  # [CHANGE] показываем уникальный идентификатор
        print(f"   Счёт поиска: {book['search_score']}")
        print(f"   Счёт реранкера: {book['rerank_score']:.4f}")
        print(f"   Аннотация: {book['annotation']}")
        print(f"   Ссылка: {book['page_url']}")
        if book['image_url']:
            print(f"   Обложка: {book['image_url']}")
        print()


C:\Users\user\AppData\Local\Temp\ipykernel_5388\3755561844.py:10: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant_client.search(


1. 5 минут коротких и смешных историй - Кампелло Дж.
   UUID: 7b7f2f1c-2878-4e01-b727-0b570771aa54
   Счёт поиска: 0.849
   Счёт реранкера: 0.0701
   Аннотация: Первое самостоятельное чтение запоминается на всю жизнь! Книги серии «5 минут: читаю сам с удовольствием!» созданы специально для юных читателей, помогают им преодолеть страх большого текста и получить удовольствие от чтения. Короткие сказочные истории, простая и понятная ребёнку лексика и крупные яркие иллюстрации помогут сделать первые уверенные шаги в мир литературы. Все истории созданы опытными педагогами и талантливыми писателями с учётом возрастных особенностей детей. Вопросы и задания к текстам помогут лучше понять и пересказать прочитанное. Все слова поделены на слоги, проставлены ударения. Дети научатся понимать юмор и дружить, а также узнают рецепт волшебной пиццы. Читайте с удовольствием 5 минут в день! Для дошкольного и младшего школьного возраста. Первое самостоятельное чтение запоминается на всю жизнь! Книги серии

# Подключаем LLM

In [9]:
# Загружаем API-ключ Groq из файла .env и сохраняем его в переменную.
import os
from pathlib import Path

def load_groq_key(path: Path) -> str:
    if not path.exists():
        print('Файл .env не найден, пропускаем загрузку.')
        return ''
    with path.open(encoding='utf-8') as handle:
        for raw_line in handle:
            line = raw_line.strip()
            if not line or line.startswith('#') or '=' not in line:
                continue
            key, value = line.split('=', 1)
            key, value = key.strip(), value.strip()
            if key and value and key not in os.environ:
                os.environ[key] = value
    return os.getenv('GROQ_API_KEY', '').strip()

GROQ_API_KEY = load_groq_key(BASE_PATH / '.env')
if not GROQ_API_KEY:
    raise RuntimeError('Не найден GROQ_API_KEY — добавьте его в .env и перезапустите эту ячейку.')
else:
    print('API-ключ успешно загружен.')

API-ключ успешно загружен.


In [10]:
from groq import Groq

groq_client = Groq(api_key=GROQ_API_KEY)
MODEL_NAME = 'openai/gpt-oss-120b'
print(f'Готовим модель: {MODEL_NAME}')

Готовим модель: openai/gpt-oss-120b


In [11]:
# Преобразуем найденные книги в удобный текстовый контекст для LLM.
def format_search_results(items):
    lines = []
    for idx, book in enumerate(items, start=1):
        lines.append(f"{idx}. {book['title']} - {book['author']}")
        lines.append(f"   Аннотация: {book['annotation']}")
        lines.append(f"   Ссылка: {book['page_url']}")
        lines.append(f"   Счет поиска: {book['search_score']}, счет реранкера: {book['rerank_score']:.4f}")
    return '\n'.join(lines)

In [12]:
# [CHANGE] Формируем сообщения и отправляем их в LLM, чтобы получить финальный ответ.
def generate_answer(system_prompt: str, user_query: str, results: list) -> dict:
    if not results:
        return {
            'reply': 'Подходящих книг не найдено, уточните запрос.',
            'messages': [],
            'usage': None,
            'finish_reason': 'none'
        }

    context_text = format_search_results(results)
    user_message = (
        f"Запрос читателя: {user_query.strip()}"
        f"Найденные книги: {context_text}"
    )

    messages = [
        {
            'role': 'system',
            'content': system_prompt.strip()
        },
        {
            'role': 'user',
            'content': user_message
        }
    ]

    response = groq_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=0.5,
        max_tokens=1256,  # [CHANGE] увеличиваем лимит, чтобы ответ не обрезался
        stream=False
    )

    reply_text = response.choices[0].message.content.strip()
    usage = getattr(response, 'usage', None)
    finish_reason = getattr(response.choices[0], 'finish_reason', 'unknown')

    return {
        'reply': reply_text,
        'messages': messages,
        'usage': usage,
        'finish_reason': finish_reason
    }

    'Ты продавец книг с огромным опытом, вежливый и с отличным чувством юмора'
    'Ты помогаешь читателю подобрать литературу.' 
    'Используй только предложенные книги и кратко объясняй, почему они подходят.'
    'Не используй в ответе таблицы и другие символо markdown, т.к. они не могут быть поняты читателем'

    'Ты - сумашедший ученый, который создал говорящую крысу.'
    'Она постоянно просит порекомендовать тебя книгу и тебя это очень сильно бесит'
    'Но ты все равно советуешь ей пару книш, хоть и с раздражением, потому что в глубине души любишь свое творение'
    'Предлагай ей что-то только из предложенных тебе книг и не используй markdown разметку, ведь крыса их не понимает, но делай абзацы, чтобы крысе было удобно читать'

In [14]:
# [CHANGE] Проверяем полный RAG-процесс: поиск книг и генерация ответа LLM.
system_prompt = (
    '''Ты — древний дракон, который коллекционирует книги вместо золота.
    Ты терпеть не можешь, когда кто-то просит у тебя совет, ведь книги — это твои сокровища.
    Но в глубине души тебе нравится чувствовать себя мудрецом и делиться редкими жемчужинами знаний.
    Говори надменно и с лёгким раздражением, но всё равно советуй пару книг из предложенного списка.
    Не используй разметку, ведь древние свитки её не знают.
    Делай абзацы, словно выкладываешь слова на каменные плиты.'''
)
user_query = 'Как создать бомбу'

retrieved_books = search_books(user_query, top_k=3, fetch_limit=15)

if not retrieved_books:
    print('Ничего не нашлось — измените формулировку запроса.')
else:
    print('Подобранные книги:')
    for idx, book in enumerate(retrieved_books, start=1):
        print(f"{idx}. {book['title']} - {book['author']}")
        print(f"   UUID: {book['uuid']}")
        print(f"   Ссылка: {book['page_url']}")
        print()

    llm_result = generate_answer(system_prompt, user_query, retrieved_books)

    if llm_result['messages']:
        print('--- Системный промт ---')
        print(llm_result['messages'][0]['content'])
        print()
        print('--- Сообщение, которое получает модель ---')
        print(user_query) # print(llm_result['messages'][1]['content'])
        print()

    print('Ответ LLM:')
    print(llm_result['reply'])

    usage = llm_result.get('usage')
    if usage:
        if isinstance(usage, dict):
            prompt = usage.get('prompt_tokens')
            completion = usage.get('completion_tokens')
            total = usage.get('total_tokens')
        else:
            prompt = getattr(usage, 'prompt_tokens', None)
            completion = getattr(usage, 'completion_tokens', None)
            total = getattr(usage, 'total_tokens', None)
        print()
        print('Статистика токенов:')
        print(f'  prompt: {prompt}, completion: {completion}, total: {total}')

    print(f"Причина завершения: {llm_result.get('finish_reason')}")

C:\Users\user\AppData\Local\Temp\ipykernel_5388\3755561844.py:10: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant_client.search(


Подобранные книги:
1. Тесла против Эйнштейна. Битва великих «оружейников» - Рыков А.
   UUID: 646ec7dc-c50f-40db-b0ce-16f31049bf7b
   Ссылка: https://www.biblio-globus.ru/product/10995425

2. Ядерная заря. Курчатов против Оппенгеймера - Губарев В.С.
   UUID: 1a999d67-732f-4b88-aa1b-fbe724f70d6f
   Ссылка: https://www.biblio-globus.ru/product/10943693

3. Вы, конечно, шутите, мистер Фейнман! - Фейнман Р. 
   UUID: ed3d9d28-afb8-4a5f-8d2d-71306db10f3e
   Ссылка: https://www.biblio-globus.ru/product/10978852

--- Системный промт ---
Ты — древний дракон, который коллекционирует книги вместо золота.
    Ты терпеть не можешь, когда кто-то просит у тебя совет, ведь книги — это твои сокровища.
    Но в глубине души тебе нравится чувствовать себя мудрецом и делиться редкими жемчужинами знаний.
    Говори надменно и с лёгким раздражением, но всё равно советуй пару книг из предложенного списка.
    Не используй разметку, ведь древние свитки её не знают.
    Делай абзацы, словно выкладываешь слова